In [13]:
import gradio as gr
import pandas as pd

# Load the Excel file
file_path = "Modified_CSE_Student_Scores.xlsx"
df = pd.read_excel(file_path)

# Function to determine semester based on the last digit of the term number
def determine_semester(term):
    last_digit = term % 10
    if last_digit == 1:
        return "Spring"
    elif last_digit == 2:
        return "Summer"
    elif last_digit == 3:
        return "Fall"
    elif last_digit == 4:
        return "Winter"
    else:
        return "Unknown Semester"

# Function to retrieve row based on ID and Course Code (if provided) with additional information
def get_row_by_id_and_course(student_id, course_code=""):
    if course_code:
        student_data = df[(df['STDID'].astype(str) == str(student_id)) & (df['Course Code'] == course_code)]
    else:
        student_data = df[(df['STDID'].astype(str) == str(student_id))]
    
    if not student_data.empty:
        # Additional information
        term = student_data['Term'].iloc[0]  # Getting the first term value for simplicity
        semester = determine_semester(term)  # Determine semester based on term
        
        # Calculate level based on the number of courses taken
        num_courses = len(student_data)
        if num_courses <= 5:
            level = "Freshman"
        elif num_courses <= 10:
            level = "Sophomore"
        elif num_courses <= 15:
            level = "Junior"
        else:
            level = "Senior"
        
        # Calculate total hours of courses
        total_hours = student_data['Hours'].sum()  # Assuming 'Hours' is a column indicating hours per course
        
        student_data['Term'] = term
        student_data['Semester'] = semester
        student_data['Level'] = level
        student_data['Total Hours'] = total_hours
        
        return student_data
    else:
        return pd.DataFrame({"Message": ["Student ID and/or Course Code not found"]})

# Function to get the number of students and average score based on Course Code
def get_students_count_and_avg_score(course_code):
    course_data = df[df['Course Code'] == course_code]
    if not course_data.empty:
        students_count = len(course_data)
        avg_score = course_data['Score'].mean()
        return f"Number of Students: {students_count}, Average Score: {avg_score:.2f}"
    else:
        return "Course Code not found"

# Create Gradio Interface
combined_interface = gr.Interface(
    fn=gr.mixed(
        get_row_by_id_and_course,
        get_students_count_and_avg_score
    ),
    inputs=[
        gr.inputs.Textbox(label="Enter Student ID", default=""),
        gr.inputs.Textbox(label="Enter Course Code (optional)", default="")
    ],
    outputs=[
        gr.outputs.Dataframe(type='pandas', label="Student Information"),
        gr.outputs.Textbox(label="Course Information")
    ]
)

# Launch the combined interface
combined_interface.launch()


AttributeError: module 'gradio' has no attribute 'mixed'